### FSR_for_force -> force

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('train_loop_config/model', ['fsr_model.LSTM', 'fsr_model.CNN_LSTM', 'fsr_model.ANN'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('train_loop_config/model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('train_loop_config/model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('train_loop_config/model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('train_loop_config/model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('train_loop_config/model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('train_loop_config/model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('train_loop_config/model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('train_loop_config/model_args/num_layer', 1, 8)
    trial.suggest_categorical('train_loop_config/criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('train_loop_config/optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('train_loop_config/optimizer_args/lr', 1e-5, 1e-1, log=True)
    trial.suggest_categorical('train_loop_config/scaler', [
        '',
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'train_loop_config/index_X': 'FSR_for_force',
        'train_loop_config/index_y': 'force'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
import ray.tune.search
import ray.tune.search.optuna
import ray_train_trainer

tuner = ray.tune.Tuner(
    trainable=ray_train_trainer.trainer,
    tune_config=ray.tune.TuneConfig(
        # scheduler=ray.tune.schedulers.ASHAScheduler(
        #     max_t=500,
        #     grace_period=1,
        #     reduction_factor=2,
        #     brackets=1,
        #     metric='rmse',
        #     mode='min',
        # ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min', 
        ),
    ), 
    # run_config=ray.air.RunConfig(
    #     callbacks=[
    #         ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
    #     ],
    # ),
) 
results = tuner.fit()

[I 2023-07-04 19:24:42,623] A new study created in memory with name: optuna
2023-07-04 19:24:44,905	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-04 19:24:46,157	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


(TorchTrainer pid=1166339) 2023-07-04 19:24:53,881	INFO backend_executor.py:137 -- Starting distributed worker processes: ['1166402 (172.26.215.93)', '1166403 (172.26.215.93)', '1166404 (172.26.215.93)', '1166405 (172.26.215.93)']
(RayTrainWorker pid=1166402) 2023-07-04 19:24:56,411	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=4]
(RayTrainWorker pid=1166402) 2023-07-04 19:24:59,307	INFO train_loop_utils.py:286 -- Moving model to device: cpu
(RayTrainWorker pid=1166402) 2023-07-04 19:24:59,308	INFO train_loop_utils.py:346 -- Wrapping provided model in DistributedDataParallel.


Trial name,date,done,experiment_tag,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
TorchTrainer_3e042081,2023-07-04_19-27-36,False,"1_criterion=torch_nn_MSELoss,index_X=FSR_for_force,index_y=force,model=fsr_model_LSTM,hidden_size=16,num_layer=3,optimizer=torch_optim_Adam,lr=0.0717,scaler=sklearn_preprocessing_RobustScaler",DESKTOP-0P789CI,269,239,9.08525e+16,172.26.215.93,1166339,416.199,True,166.155,0.713814,166.155,1688466456,269,3e042081


2023-07-04 19:27:37,358	ERROR tune_controller.py:873 -- Trial task failed for trial TorchTrainer_3e042081
Traceback (most recent call last):
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 18, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/_private/worker.py", line 2540, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::_Inner.train() (pid=1166339, ip=172.26.215.93, actor_id=075163b0975cca3f59fbc05401000000, repr=TorchTrainer)
  File "/home/seokj/workspace/.venv/lib/python3.10/site-packages/ray/tune/tr